<a href="https://colab.research.google.com/github/Guo-bot-1998/Appendicitis/blob/master/Appendicitis_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [1]:
!pip install timm
!pip install tqdm
!pip install kora
!pip install fake_useragent

In [9]:
import nibabel as nib
import numpy as np
import os
import pandas as pd
import nibabel as nib
import torch
import torch.nn as nn
import torch.optim as optim
import random
import re
import json
import tqdm
import timm
import gdown
from kora.xattr import get_id
from torch.utils.data import Dataset, DataLoader
import requests
from fake_useragent import UserAgent
import time

## 存取

In [82]:

def process_data2(shift=0, termi=10, dropFalse=0.85, dropPos=0)\
 -> tuple[torch.tensor, torch.tensor, list[tuple[str, int]]]:
  """Reading data in "datadir" and process them.
  Taking care of memory efficiency"

  Args:
    shift: 從第幾個開始讀取
    termi: 讀取幾個
    dropFalse: 被刪除的有闌尾炎的張數比例
    dropPos: 被刪除的有闌尾炎的張數比例

  Returns:
    images: 输出的图像数据
    labels: 输出的标签数据
    scans_info: 每個scan的檔名和切片數(原本nii檔所包含的數量)
  """

  labels_ = read_label(labelpath)

  if labels_.index.name != 'id':
    labels_.set_index('id', inplace=True)

  #裁減過的圖片放在Cropped_[範圍]的資料夾下,用路徑名稱判斷資料使否裁減過
  cropmatch = re.search(r'Cropped', datadir)
  if not cropmatch:
    print("讀取未切片的資料夾")
    xlim, ylim, zlim = [0,512], [0,512], [0,None]
  else:
    #範圍 = xstart-xend_ystart-yend_zstart-zend
    ismatch = re.search(r'(\d+-\d+)_(\d+-\d+)_(\d+-\d+)', string)
    if ismatch:
      dimens = ismatch.group(0)
      labelidx = dimens.split('_')
      xlim = [int(idx) for idx in labelidx[0].split('-')]
      ylim = [int(idx) for idx in labelidx[1].split('-')]
      zlim = [int(idx) for idx in labelidx[2].split('-')]
      print(f"{xlim=}\n{ylim=}\n{zlim=}")
      if xlim[0] >= xlim[1] or ylim[0] >= ylim[1] or zlim[0] >= zlim[1]:
        print("切片格式不正确")
        return
    else:
      print("切片格式不正确")
      return


  filelist = os.listdir(datadir)
  filelist = filelist[shift:shift+termi]
  scans_info = []

  # preallocated
  if not cropmatch:
    selecteds = [labels_.loc[labels_.index.str.startswith(afile.strip('.nii.gz')+'_')] for afile in filelist]
    numcuts = [len(selected) for selected in selecteds]
    numcuts = sum(numcuts)
  else:
    numcuts = (zlim[1]-zlim[0]) * len(filelist)
  images = torch.zeros(numcuts,1, ylim[-1]-ylim[0],xlim[-1]-xlim[0])
  labels = -torch.ones(numcuts)
  nimgs = 0
  nprocess = 0

  for key in filelist:
    file_path = os.path.join(datadir, key)
    key = key.strip('.nii.gz')
    scan  = labels_.loc[labels_.index.str.startswith(key+'_')]

    value = nib.load(file_path).get_fdata()
    scans_info.append((key, value.shape[2]))


    label_t = torch.tensor(scan['label'][zlim[0]:zlim[-1]])
    image_t = torch.from_numpy(value).float().permute(2, 0, 1).unsqueeze(1)

    #有幾筆不是512x512
    if (not cropmatch) and (image_t.shape[2] != 512 or image_t.shape[3] != 512):
      image_t = cropping(image_t)

    image_t, label_t = remove_false_images(image_t, label_t, dropFalse)
    image_t, label_t = remove_positive_images(image_t, label_t, dropPos)

    n_new = len(label_t)

    images[nimgs:nimgs+n_new] = image_t
    labels[nimgs:nimgs+n_new] = label_t
    nimgs += len(label_t)
    print(f"Process {nprocess}: {key} finished...")
    nprocess += 1

  print(f"read {len(scans_info)} scans")

  return images, labels.float(), scans_info




def save_model(model, modelname="", dirname="", root=os.path.realpath("/content/drive/MyDrive/AOCR2024/params")):

  if not model:
    print("not given model")
    return

  if not os.path.exists(root):
    print(f"{root} not exists!")
    return

  print(f"model will be saved under {root}")


  if not modelname:
    modelname = input("請輸入模型儲存的檔名:")
  if not dirname:
    dirname = input("請輸入模型儲存的資料夾:")

  filename = f"{root}/{dirname}/{modelname}"

  if not os.path.exists(os.path.dirname(filename)):
    os.mkdir(os.path.dirname(filename))

  if os.path.isfile(filename+'.pth'):
      print(f"{filename}.pth exist.")
  else:
      torch.save(model.state_dict(), f'{filename}.pth')

  return filename



def read_label(excel_path) -> pd.DataFrame:
  """Reads a csv file containing ground-truth.
    The csv file should have two columns: 'id' and 'label'.
  """
  with open(excel_path, 'r') as f:
    df = pd.read_csv(f)
    df.set_index('id', inplace=True)
    return df



def write_error(logpath, e, nprocess):
  # with open(logpath, 'a') as f:
    # f.write(f"{nprocess:} error occured: {e}\n") ##! may stuck the program!!
  print(f"{nprocess:} error occured: {e}\n")


def read_submission(excel_path) -> pd.DataFrame:
  """Reads a csv file containing submission file.
    The csv file should follow the format given by Kaggle.
  """
  with open(excel_path, 'r') as f:
    df = pd.read_csv(f)
    df.set_index('id', inplace=True)
    return df




## 列印結果

In [4]:
def get_confusion(guess, truth):
  right = guess == truth
  wrong = np.logical_not(right)
  TP = np.sum(np.logical_and(right, truth == np.ones(right.shape)))
  TN = np.sum(np.logical_and(right, truth == np.zeros(right.shape)))
  FN = np.sum(np.logical_and(wrong, truth == np.ones(wrong.shape)))
  FP = np.sum(np.logical_and(wrong, truth == np.zeros(wrong.shape)))
  return (TP,FP,FN,TN)



def print_results(prediction, labels):
  if torch.is_tensor(prediction):
    prediction = prediction.cpu().numpy()
  if torch.is_tensor(labels):
    labels = labels.cpu().numpy()

  TP,FP,FN,TN = get_confusion(prediction, labels)
  print(f"\n\
      真實值\n\
  預  +-----+-----+\n\
  測| TP: {TP}| FP: {FP}|\n\
  值| FN: {FN}| TN: {TN}|\n\
      +-----+-----+ \n")

  recall = TP/(TP + FN)
  precision = TP/(TP + FP)
  recall = 0 if np.isnan(recall) else recall.item()
  precision = 0 if np.isnan(precision) else precision.item()
  F1 = 0 if recall + precision == 0 else  (2*recall*precision/(recall+precision))

  print(f"{recall=}\n{precision=}\n{F1=}\n")

## 數據處理




In [5]:
def argumenting(images, labels, n=5):
  """
  數據增強。
  在images裡面把所有有闌尾炎的cut複製給定次數，並
  插入回images的隨機位置裡面

  Args:
    images: 输入的图像数据
    labels: 输入的标签数据
    n: 每个样本複製多少次

  Returns:
    增強后的图像数据和标签数据
  """
  where = (labels == True).nonzero(as_tuple=True)[0]
  bad_images = images[where]

  rep_imgs = bad_images.repeat(n, 1, 1, 1)
  rep_labels = torch.ones(rep_imgs.shape[0], dtype=labels.dtype)

  # import pdb
  # pdb.set_trace()
  # 隨機插入argumented圖片
  nimg = images.shape[0]
  rnd_pos = torch.randint(0, nimg, (rep_imgs.shape[0],))
  images = torch.cat((images, rep_imgs), dim=0)
  images = images[torch.argsort(torch.cat((torch.arange(nimg), rnd_pos)))]

  # 對應對置插入標籤
  labels = torch.cat((labels, rep_labels), dim=0)
  labels = labels[torch.argsort(torch.cat((torch.arange(nimg), rnd_pos)))]

  return images, labels

def remove_false_images(images, labels, ratio):
    """
    隨機在images裡面移除一定比例的無闌尾炎cut

    Args:
      images: 输入的图像数据
      labels: 输入的标签数据
      ratio: 移除的比例

    Returns:
      移除后的图像数据和标签数据
    """
    # 找出 labels == 0 的索引
    where_false = (labels == 0).nonzero(as_tuple=True)[0]
    mask = torch.ones(len(images), dtype=torch.bool)
    indices_filter = torch.randperm(len(where_false))[:int(len(where_false)*ratio)]
    mask[where_false[indices_filter]] = False
    images = images[mask]
    labels = labels[mask]

    return images, labels

def remove_positive_images(images, labels, ratio):
    # 找出 labels == 1 的索引
    where_positive = (labels == 1).nonzero(as_tuple=True)[0]
    mask = torch.ones(len(images), dtype=torch.bool)
    indices_filter = torch.randperm(len(where_positive))[:int(len(where_positive)*ratio)]
    mask[where_positive[indices_filter]] = False
    images = images[mask]
    labels = labels[mask]

    return images, labels

In [28]:
import re

# 示例字符串
string = "/content/drive/MyDrive/AOCR2024/Cropped_60-316_150-406_11-71/Train_Valid_Image_cropped/Zx00AD16F8B97A53DE6E7CFE260BDF122F0E655659A3DF1628.nii.gz"

# 正则表达式匹配模式
# 这个模式匹配三组“数字-数字”，每组之间由下划线分隔
pattern = r'(\d+-\d+)_(\d+-\d+)_(\d+-\d+)'

# 使用正则表达式进行匹配
match = re.search(pattern, string)

if match:
    dimens = match.group(0)
    labelidx = dimens.split('_')[-1].split('-')
    print("提取的部分:", labelidx)
else:
    print("没有找到匹配")


提取的部分: ['11', '71']


## 其他

In [6]:

def count_zero(images):
  count = 0
  for image in images:
    if not torch.any(image):
      count += 1
  return count

def cropping(image):
  if image.shape[2] != 512:
    start = (image.shape[2] - 512) // 2
    end = start + 512
    image = image[:, :, start:end, :]

  if image.shape[3] != 512:
    start = (image.shape[3] - 512) // 2
    end = start + 512
    image = image[:, :, :, start:end]
  return image




def custom_sort_key(val):
    parts = val.split('_')
    if len(parts) == 2 and parts[1].isdigit():
        return (parts[0], int(parts[1]))
    return (parts[0], -1)  # 使沒有_(數字)的id排最前面


def isgpu():
    """檢查是否有 CUDA 支持的 GPU"""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("GPU is available")
    else:
        device = torch.device("cpu")
        # raise("GPU not available")
    return device


In [7]:
device = isgpu()

# 掛載


In [8]:
from google.colab import drive
drive.mount('/content/drive/',)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pwd

/content


# 資料下載

In [80]:
drive_rootdir = "/content/drive/MyDrive/AOCR2024"
labelpath = '/content/drive/MyDrive/AOCR2024/TrainValid_ground_truth.csv'

datadir ="Cropped_60-316_150-406_11-71/Train_Valid_Image_cropped"
if not os.path.exists('/content/' +datadir):
  os.mkdir(datadir)


In [ ]:
!curl ipecho.net/plain

34.80.136.131

In [ ]:


# user_agents = [
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",  # Chrome on Windows 10
#     "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",  # Safari on macOS
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0",  # Firefox on Windows 10
#     "Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",  # Safari on iPhone
#     "Mozilla/5.0 (Linux; Android 10; SM-G973F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.210 Mobile Safari/537.36",  # Chrome on Android
#     "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"  # Firefox on Ubuntu Linux
# ]
# headers = {
#     'User-Agent': random.choice(user_agents)
# }
ua = UserAgent()

files_on_drive = os.listdir(drive_rootdir+'/'+datadir)
files_in_localdir = os.listdir(f"/content/{datadir}")
files_to_download = [f for f in files_on_drive if f not in files_in_localdir]
files = [(filename, get_id(f'{drive_rootdir}/{datadir}/{filename}')) for filename in files_to_download]
# print(headers)

t = random.randint(20,30)

with requests.Session() as session:
  for idx, (name, id) in enumerate(files):
    headers = {'User-Agent': ua.random}
    # print(headers)
    url = f"https://drive.google.com/uc?id={id}&confirm=t"
    # url = f"https://drive.google.com/file/d/{id}/view?usp=drive_link"
    # url = f"https://drive.google.com//uc?id={id}&confirm=t&usp=sharing"
    response = session.get(url, headers=headers, allow_redirects=True)
    if response.status_code == 200:
        with open(f"/content/{datadir}/{name}", 'wb') as file:
            file.write(response.content)
        print(f"{idx+1}/{len(files)}: {name}")
    elif response.status_code == 403:
        print(f"Error: Unable to download file. Status code: {response.status_code}")
        # headers = {
        #   'User-Agent': random.choice(user_agents)
        # }
        # print(headers)
        s = random.randint(300,600)
        print(f"sleeping {s} secondes...")
        time.sleep(s)
    else:
        print(f"Error: Unable to download file. Status code: {response.status_code}")

    # if (idx+1) % t  == 0:
    #   s = random.randint(60,300)
    #   print(f"sleeping {s} secondes...")
    #   time.sleep(s)
    #   t = random.randint(20,30)

# files_on_drive = os.listdir(drive_rootdir+'/'+datadir)
# files_in_localdir = os.listdir(f"/content/{datadir}")
# files_to_download = [f for f in files_on_drive if f not in files_in_localdir]
# files = [(filename, get_id(f'{drive_rootdir}/{datadir}/{filename}')) for filename in files_to_download]
# for idx, (name, id) in enumerate(files):
#   gdown.download(f"https://drive.google.com/uc?id={id}&confirm=t", output=f"/content/{datadir}/{name}", quiet=True, use_cookies=False)
#   print(f"{idx+1}/{len(files)}: {name}")

In [61]:
len(os.listdir(datadir))

40

In [ ]:
# # 測試直接從mount下載
# labelpath = '/content/drive/MyDrive/AOCR2024/TrainValid_ground_truth.csv'
# datadir ="/content/drive/MyDrive/AOCR2024/Train_Valid_Image"

# process_data2(termi=5,shift=0,dropFalse=0,dropPos=0)

In [91]:
# # 測試下載
# labelpath = '/content/drive/MyDrive/AOCR2024/TrainValid_ground_truth.csv'

# dflabel = read_label(labelpath)
# images, labels, _ =  process_data2(termi=3,shift=2,dropFalse=0,dropPos=0)

xlim=[60, 316]
ylim=[150, 406]
zlim=[11, 71]
Process 0: Zx0AA2C956EEA5EAEA0DC0DA2606A8F8D0F334B97FDD860E1F finished...
Process 1: Zx08DA48B45B5C7D0182C369B8E0B013CAF9EEA43199E8AFEF finished...
Process 2: Zx0800DE5C96380322E442453B99E25D6F1282F2C4A610A497 finished...
read 3 scans


In [95]:
# s1 = labels[0:60]
# s2 = labels[60:120]
# s3 = labels[120:180]
# s2

# 資料處理

In [45]:
labelpath = '/content/drive/MyDrive/AOCR2024/TrainValid_ground_truth.csv'

dflabel = read_label(labelpath)
total_nii = len(os.listdir(datadir))
images, labels, _ =  process_data2(termi=total_nii,shift=0,dropFalse=0,dropPos=0)

xlim=[60, 316]
ylim=[150, 406]
zlim=[11, 71]
Process 0: Zx0B06041DB37FAF0160B564B48791D3B58804B731DD57F4AB finished...
Process 1: Zx0CA5CE197AA9378EE23F949BDFC91790D4FED5D6FD06F5A2 finished...
Process 2: Zx0AA2C956EEA5EAEA0DC0DA2606A8F8D0F334B97FDD860E1F finished...
Process 3: Zx08DA48B45B5C7D0182C369B8E0B013CAF9EEA43199E8AFEF finished...
Process 4: Zx0800DE5C96380322E442453B99E25D6F1282F2C4A610A497 finished...
Process 5: Zx0BCA7F5108A7E8BCC2AB6094DE20A5C243897FCB0C82E20A finished...
Process 6: Zx052D253409F40A03B71D313382692559FB09F771A41F754A finished...
Process 7: Zx00AD16F8B97A53DE6E7CFE260BDF122F0E655659A3DF1628 finished...
Process 8: Zx00FE1B9A88E88C71F8D11F50C2B5FBFEB3461E67BE1E83B8 finished...
Process 9: Zx04424F2CF83E0AD3DDF3CC99A41482D909CBC495238F2FEC finished...
Process 10: Zx0CE3BA2363A72325A7BB82B688463DBB79D224C67883F062 finished...
Process 11: Zx07C5208D04A267A61BFB6EC968FBBB3DF25C245A6893273D finished...
Process 12: Zx014E4DF04789E7E61C3B4876CE4D122DC14E0BB842E42E7D fi

In [ ]:
# %reset_selective images

In [ ]:
# %reset

# 訓練 (EfficiencyNetV2_m)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [ ]:
model_name = "tf_efficientnetv2_m"
pretrained_model = timm.create_model(model_name, pretrained=True)

# 修改輸入通道
pretrained_model.conv_stem = nn.Conv2d(1, 24, kernel_size=3, stride=2, padding=1, bias=False)

# 修改輸出類別
num_classes = 1
pretrained_model.classifier = nn.Linear(pretrained_model.classifier.in_features, num_classes)

# 添加 Sigmoid 激活函數
pretrained_model = nn.Sequential(
    pretrained_model,
    nn.Sigmoid()
)

# 檢查模型結構
# print(pretrained_model)


In [ ]:
def between_epoch():
  del images, labels
  images, labels, _ = process_data2(termi=total_nii, shift=0,dropFalse=0.8, dropPos=0)
  dataset = CustomDataset(images, labels)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
  return dataloader

In [ ]:
# 訓練參數
num_epochs = 10
batch_size = 8
lr = 0.01
running_loss_list = []

# 初始化模型、損失函數和優化器
model = pretrained_model
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr)
model = model.to(device)

dataset = CustomDataset(images, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_images, batch_labels in dataloader:
        batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch_images)
        outputs = outputs.squeeze()
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    dataloader = between_epoch()

    running_loss_list.append(running_loss/len(dataloader))
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}")

## Loss圖

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(1,num_epochs+1)[:len(running_loss_list)], running_loss_list)
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# 儲存模型參數



In [ ]:
filename = save_model(model)

params = {
    'num_epochs': num_epochs,
    'batch_size': batch_size,
    'learning_rate': lr,
}

if os.path.isfile(filename+'.json'):
    print(f"{filename}.json exist.")
else:
    with open(f'{filename}.json', 'w') as f:
        json.dump(params, f)

# 讀取模型參數

In [ ]:
# model = pretrained_model
# model = model.to(device)

In [ ]:
import json
filename = input("請輸入要獲取模型路徑:")

if not os.path.isfile(filename+'.pth'):
    print(f"{filename}.pth not exist.")
else:
    print(model.load_state_dict(torch.load(filename+'.pth')))

with open(f'{filename}.json', 'r') as f:
    params = json.load(f)


# 評估


In [ ]:
data = read_data("TrainValid_Image/train_data",termi=80,shift=80)
# data = read_data("Test1_Image/test_data",termi=3,shift=1)
dflabel = read_label("TrainValid_ground_truth.csv")

In [ ]:
images,labels,info = process_data(data, dflabel)

In [ ]:
# 評估設置
num_epochs = params['num_epochs']
batch_size = params['batch_size']

#最後data不滿一個batch
num_batches = len(images) // batch_size
if len(images) % batch_size != 0:
  num_batches += 1

In [ ]:
model.eval()

predict_list = torch.tensor([]).to(device)
with torch.no_grad():  # 不更新梯度
    correct = 0
    total = 0
    for i in range(num_batches):
        batch_images = images[i*batch_size:(i+1)*batch_size].to(device)
        batch_labels = labels[i*batch_size:(i+1)*batch_size].to(device)

        outputs = model(batch_images)
        predicted = (outputs.squeeze() > 0.5).int()
        if predicted.dim() == 0:
          predicted = predicted.unsqueeze(0)
        predict_list = torch.cat((predict_list,predicted),0)


predict_listq = predict_list.cpu()



In [ ]:
print_results(predict_listq, labels[:len(predict_list)])

# 輸出至submission.csv

In [ ]:
def cont(l):
  "1是否連續"
  f1 = False
  f2 = False
  for i in range(len(l)):
    if (l[i] == 1):
      f1 = True
    if (l[i] == 0):
      if (f1):
        f2 = True
    if (l[i] == 1) and f2:
      return False
  return True

def yes(predict):
  return int(sum(predict) >= 3)



predict_list = predict_listq.numpy()

## 還原id與對應label，假設key按照scan輸入順序排列,每個key對應的scan的cuts數是nslice
output = {}
k = 0  #第幾個scan
ii = 0   #每個key輪到第幾個
id, nslice = info[k][0], info[k][1]
for i in range(len(predict_list)):

  if (ii >= nslice):
    #該換下一個scan了
    output[id] = yes(predict_list[i-nslice:i]) #評估方式

    k += 1
    ii = 0
    id, nslice = info[k][0], info[k][1]

  label = predict_list[i]
  output[id+f'_{ii}'] = int(predict_list[i])
  ii += 1

output[id] = yes(predict_list[(i+1)-ii:]) #補上最後一個scan評估
# import pdb
# pdb.set_trace()
output = list(output.items())
dfout = pd.DataFrame(output)
dfout.columns = ['id', 'label']
dfout = dfout.sort_values(by='id', key=lambda x: x.map(custom_sort_key))
filename = input("輸入提交檔名(enter for submission)")
if filename == '':
  filename = 'submission'
dfout.to_csv(filename+'.csv', index=False)

In [ ]:
info

In [ ]:
dfout

# 查看submission.csv

In [ ]:
!pwd

In [ ]:
dftest = read_submission('fisrt_80.csv')
# dftest = read_submission('submission.csv')
dflabel = read_label("TrainValid_ground_truth.csv")

In [ ]:
len(dftest)

In [ ]:
import re


# 抓出scan-level data
pattern = re.compile(r'.*_[0-9]+$')  # 正則表達式匹配 "_數字" 結尾
mask = ~dftest.index.str.match(pattern)
scan_guess = np.array(dftest[mask]['label'])
scan_truth = np.array(dflabel.loc[dftest[mask]['label'].index]['label'])

mask = ~mask
cut_guess = np.array(dftest[mask]['label'])
cut_truth = np.array(dflabel.loc[dftest[mask]['label'].index]['label'])

In [ ]:
ss = 10
print("="*ss + "F1 score on scan level" + "="*ss)
print_results(scan_guess, scan_truth)
print(),print()
print("="*ss + "F1 score on cut level" +"="*ss )
print_results(cut_guess, cut_truth)

# 其他指令
不在工作流
當參考

In [ ]:
import time
datadir = 'Train_Valid_Image'
for idx, afile in enumerate(os.listdir(datadir)):
  file_path = os.path.join(datadir, afile)
  time.sleep(1)
  nii_file =  nib.load(file_path)
  print(f"{idx}: read {afile}")

In [ ]:
pattern = re.compile(r'.*_[0-9]+$')  # 正則表達式匹配 "_數字" 結尾
mask = dftest.index.to_series().str.match(pattern)
dflabel_ = dftest[mask]
dflabel_.index.map(lambda x : x.split('_')[1]).sort_values()[-3:]

In [ ]:
import random

# 假設這是您的列表
my_list = np.array([1, 2, 3, 4])

mask = [True, False, False, True]
my_list[mask]

In [ ]:
datadir ="TrainValid_Image/train_data"
labelpath = 'TrainValid_ground_truth.csv'
labels_ = read_label(labelpath)


filelist = os.listdir(datadir)
filelist = filelist[10:20]
selecteds = [labels_.loc[labels_.index.str.startswith(afile.strip('.nii')+'_')] for afile in filelist]
scans_info = []
# preallocated
print("enter")
numcuts = [len(selected) for selected in selecteds]
numcut = sum(numcuts)

In [ ]:
numcuts = [len(selected) for selected in selecteds]

In [ ]:
img = nib.load('Train_Valid_Image/Zx00AD16F8B97A53DE6E7CFE260BDF122F0E655659A3DF1628.nii.gz')


In [ ]:
drive.flush_and_unmount()

In [ ]:
t1 = torch.randint(3,(5,1,2,2))
lab = torch.tensor([1,1,1,1,1])
t1

In [ ]:
remove_positive_images(t1,lab,0.79)

In [ ]:
!pip install -U --no-cache-dir gdown gdown

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown https://drive.google.com/drive/folders/1C7HXpHMw1Alvwif9hO97FUzfn4rhxG8B -O Train_Valid_Image --folder --remaining-ok